<a href="https://colab.research.google.com/github/zdwhite/Thinkful-Unit-3/blob/master/KNN_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> Why don't you **add the other feature** and **mess around with  k**  and **weighting** to see if you can do any better than we've done so far?



In [8]:
!pip install --upgrade matplotlib

Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (3.0.2)


Restart runtime

In [0]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

music = pd.DataFrame()
music['duration'] = [184, 134, 243, 186, 122, 197, 294, 382, 102, 264, 
                     205, 110, 307, 110, 397, 153, 190, 192, 210, 403,
                     164, 198, 204, 253, 234, 190, 182, 401, 376, 102]
music['loudness'] = [18, 34, 43, 36, 22, 9, 29, 22, 10, 24, 
                     20, 10, 17, 51, 7, 13, 19, 12, 21, 22,
                     16, 18, 4, 23, 34, 19, 14, 11, 37, 42]
music['bpm'] = [ 105, 90, 78, 75, 120, 110, 80, 100, 105, 60,
                  70, 105, 95, 70, 90, 105, 70, 75, 102, 100,
                  100, 95, 90, 80, 90, 80, 100, 105, 70, 65]

## Include Duration

### Non-Weighted

In [46]:
from sklearn import neighbors

# Build our model.
knn = neighbors.KNeighborsRegressor(n_neighbors=10)
X = pd.DataFrame(music[['loudness','duration']])
Y = music[['bpm']]
knn.fit(X, Y)



KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=10, p=2,
          weights='uniform')

In [47]:
print('\nR-squared:')
print(knn.score(X, Y))


R-squared:
0.09322387776406926


Here is the R<sup>2</sup> for 10 neighbors, including our other feature.



####Cross Validation

In [48]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))


Unweighted Accuracy: -0.42 (+/- 0.78)


Looks like we got worse at predicting.

### Weighted

In [49]:
from sklearn import neighbors

# Build our model.
knn = neighbors.KNeighborsRegressor(n_neighbors=10,weights='distance')
X = pd.DataFrame(music[['loudness','duration']])
Y = music[['bpm']]
knn_w.fit(X, Y)

print('\nR-squared:')
print(knn.score(X, Y))


R-squared:
0.9928822245420897


Holy 8 ball batman our model can predict everything

Careful Robin that model is no fortune teller it's overfitting! Stand back I'll show you.

#### Cross Validation

In [52]:
from sklearn.model_selection import cross_val_score

score_w = cross_val_score(knn_w, X, Y, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Weighted Accuracy: -0.51 (+/- 1.24)


## Mess with all the Parameters!

It was brought to my attention that SKlearn's gridsearch function is perfectly suited for this task

In [73]:

from sklearn.model_selection import GridSearchCV
X = pd.DataFrame(music[['loudness','duration']])
Y = music[['bpm']]

knn = neighbors.KNeighborsRegressor()

parameters = {
    'n_neighbors':np.arange(1, 21, 1),
    'weights':['uniform','distance']
}

grid = GridSearchCV(knn, parameters, scoring='r2', cv=5, verbose=0)
#Fit the Data
grid.fit(X, Y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]), 'weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [74]:
grid.best_params_

{'n_neighbors': 17, 'weights': 'distance'}

In [98]:
grid.cv_results_['mean_test_score'][grid.best_index_], grid.cv_results_['std_test_score'][grid.best_index_]

(-0.22260040335155942, 0.28285600182106746)

Well this sucks, our best itteration of the model doesn't predict very well.

In [102]:
from sklearn.model_selection import GridSearchCV
X = pd.DataFrame(music.loudness)
Y = music[['bpm']]

knn = neighbors.KNeighborsRegressor()

parameters = {
    'n_neighbors':np.arange(1, 21, 1),
    'weights':['uniform','distance']
}

grid = GridSearchCV(knn, parameters, scoring='r2', cv=5, verbose=0)
#Fit the Data
grid.fit(X, Y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]), 'weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [103]:
grid.best_params_

{'n_neighbors': 6, 'weights': 'distance'}

In [105]:
grid.cv_results_['mean_test_score'][grid.best_index_], grid.cv_results_['std_test_score'][grid.best_index_]

(0.16767917386506967, 0.411929287645543)

While this also sucks, it seems that duration had a negative effect on prediction.